I've been working with [`spacy`](https://spacy.io/) more and more over the years, and I thought it'd be a good idea to write about pieces of the configuration system. There are mentions of it throughout the [docs](https://spacy.io/usage/training#config) and in some of the `spacy` 3.0 [videos](https://youtu.be/BWhh3r6W-qE), but I have yet to find a super detailed breakdown of what's going on—the closest being this [blog](https://explosion.ai/blog/spacy-v3-project-config-systems#spacy-config-system). This post will hopefully shed some light on the components that [*share* or *listen to*](https://spacy.io/usage/embeddings-transformers#embedding-layers) previous components in the pipeline.

Let's start with a brief demo of `spacy`.

> Install `spacy` and the `en_core_web_sm` model if you want to follow along:
> ```shell
$ pip install spacy
$ python -m spacy download en_core_web_sm
```

In [1]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("Hi, my name is Ian and this is my blog.")
print(doc)

Hi, my name is Ian and this is my blog.


Nothing fancy on the surface, but this [`doc`](https://spacy.io/api/doc) object that we've created is the product of sending our string of characters through a [pipeline of models](https://spacy.io/usage/processing-pipelines), or as `spacy` likes to call them, [components](https://spacy.io/usage/processing-pipelines#pipelines). We can view the pipeline components via the [`nlp.pipeline` property](https://spacy.io/api/language#attributes).

In [2]:
nlp.pipeline

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x1ed788cbc50>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x1ed78a741d0>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x1ed787cd620>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x1ed7890f5d0>),
 ('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x1ed78919050>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x1ed787cd310>)]

And we can get more component information with [`nlp.analyze_pipes`](https://spacy.io/api/language#analyze_pipes) such as what each assigns, their requirements, their scoring metrics, whether they retokenize, and in what order the components perform their annotations.

In [3]:
# note the semicolon (;) to reduce output after the table.
nlp.analyze_pipes(pretty=True);


============================= Pipeline Overview =============================

#   Component         Assigns               Requires   Scores             Retokenizes
-   ---------------   -------------------   --------   ----------------   -----------
0   tok2vec           doc.tensor                                          False      
                                                                                     
1   tagger            token.tag                        tag_acc            False      
                                                                                     
2   parser            token.dep                        dep_uas            False      
                      token.head                       dep_las                       
                      token.is_sent_start              dep_las_per_type              
                      doc.sents                        sents_p                       
                                                       sents

Notice the first component, `tok2vec`. This [component](https://spacy.io/api/tok2vec) is responsible for mapping tokens to vectors, i.e., creating an [embedding layer](https://spacy.io/usage/embeddings-transformers), and making them available for later components to use via the `doc.tensor` attribute.
> Note, this is not the same as a [`tokenizer`](https://spacy.io/api/tokenizer).

In the [`en_core_web_sm`](https://spacy.io/models/en#en_core_web_sm) pipeline, we can see that the [`tagger`](https://spacy.io/api/tagger) and [`parser`](https://spacy.io/api/dependencyparser) components both use the `tok2vec`'s output by accessing the `tok2vec.listening_components`.

In [4]:
tok2vec = nlp.get_pipe("tok2vec")
tok2vec.listening_components

['tagger', 'parser']

On the flip side, we can see which components *use* a `tok2vec` model by checking their configurations via `nlp.get_pipe_config`.

In [5]:
[
    name
    for name in nlp.pipe_names
    if (model := nlp.get_pipe_config(name).get("model")) is not None
    and model.get("tok2vec") is not None
]

['tagger', 'parser', 'ner']

The `tagger` and `parser` are both present as expected, but so is the `ner` component which has its own `tok2vec` layer, separate from the `tok2vec` at the beginning of the `nlp.pipeline`.

In [6]:
ner_tok2vec = nlp.get_pipe_config("ner")["model"]["tok2vec"]
ner_tok2vec

{'@architectures': 'spacy.Tok2Vec.v2',
 'embed': {'@architectures': 'spacy.MultiHashEmbed.v2',
  'width': 96,
  'attrs': ['NORM', 'PREFIX', 'SUFFIX', 'SHAPE'],
  'rows': [5000, 1000, 2500, 2500],
  'include_static_vectors': False},
 'encode': {'@architectures': 'spacy.MaxoutWindowEncoder.v2',
  'width': 96,
  'depth': 4,
  'window_size': 1,
  'maxout_pieces': 3}}

This is an example of an *independent* component—it can stand alone without the other other components.

The `tagger` and `parser` components both *listen to* or *share* the `tok2vec` component's output in the `nlp.pipeline`.

In [7]:
tagger_tok2vec = nlp.get_pipe_config("tagger")["model"]["tok2vec"]
tagger_tok2vec

{'@architectures': 'spacy.Tok2VecListener.v1',
 'width': '${components.tok2vec.model.encode:width}',
 'upstream': 'tok2vec'}

In [8]:
parser_tok2vec = nlp.get_pipe_config("parser")["model"]["tok2vec"]
parser_tok2vec

{'@architectures': 'spacy.Tok2VecListener.v1',
 'width': '${components.tok2vec.model.encode:width}',
 'upstream': 'tok2vec'}

Listening to/sharing an upstream component has some pros and cons including speed and flexibility (see my [stack overflow answer](https://stackoverflow.com/a/76774652/6509519) for an experiment). Sometimes sharing a component can help boost later components metrics, and other times it's easier to have something more independent.

When it comes to training components that listen to the same `tok2vec` layer, we have options.
- We could add a component with its own `tok2vec` similar to the `ner` component.
- We could add a component and have it listen to the existing `tok2vec` layer.
- We could add both a component and have it listen to a *new* `tok2vec` component, separate from the existing one (rarely would you want to do this).

Here is an example of the first option—we'll enable the [`senter`](https://spacy.io/api/sentencerecognizer) model (which is pre-trained so we won't need to actually train it), and view its `tok2vec` setup.

> Note, you could do this with a custom component as well assuming it's registered or in the environment.

In [9]:
nlp.enable_pipe("senter")
senter_tok2vec = nlp.get_pipe_config("senter")["model"]["tok2vec"]

senter_tok2vec

{'@architectures': 'spacy.Tok2Vec.v2',
 'embed': {'@architectures': 'spacy.MultiHashEmbed.v2',
  'width': 16,
  'attrs': ['NORM', 'PREFIX', 'SUFFIX', 'SHAPE', 'SPACY'],
  'rows': [1000, 500, 500, 500, 50],
  'include_static_vectors': False},
 'encode': {'@architectures': 'spacy.MaxoutWindowEncoder.v2',
  'width': 16,
  'depth': 2,
  'window_size': 1,
  'maxout_pieces': 2}}

Pretty easy to do. If we wanted something more like the second option, we'd need to update the [`nlp.config`](https://spacy.io/api/language#config) telling `spacy` that we want the `senter` to listen to the existing `tok2vec` component.

In [10]:
# Path extracted from ``parser_tok2vec`` -> ${components.tok2vec.model.encode:width}
width = tok2vec.model.get_ref("encode").get_dim("nO")

senter_config = {
    "model": {
        "tok2vec": {
            "@architectures": "spacy.Tok2VecListener.v1",
            "width": width,
            "upstream": "tok2vec",
        }
    },
}

# Before adding ``senter`` with listener.
tok2vec.listening_components

['tagger', 'parser']

In [11]:
# Removing existing ``senter`` model without listener.
nlp.remove_pipe("senter")

# Adding new ``senter`` model with listener.
# nlp.add_pipe("senter", after="parser", config=senter_config)
nlp.add_pipe("senter", after="parser", config=senter_config)

# After adding ``senter`` with listener.
tok2vec.listening_components

['tagger', 'parser', 'senter']

In [12]:
nlp.get_pipe_config("senter")["model"]["tok2vec"]

{'@architectures': 'spacy.Tok2VecListener.v1',
 'width': 96,
 'upstream': 'tok2vec'}

The `senter` component has its own `tok2vec` layer just like the `ner` component. If we wanted to have it listen to the same `tok2vec` as the `tagger` and `parser` we could add a listner to the `tok2vec` using the `tok2vec.add_listener` method. This requires us to create a `Tok2VecListener` instance.

In [10]:
from spacy.pipeline.tok2vec import Tok2VecListener

# Taking the width from the current `tok2vec` config
width = nlp.get_pipe_config("tok2vec")["model"]["encode"]["width"]
tok2vec_listener = Tok2VecListener(upstream_name="*", width=width)

# Current listeners
tok2vec.listening_components

['tagger', 'parser']

In [11]:
tok2vec.add_listener(listener=tok2vec_listener, component_name="senter")

# New listeners
tok2vec.listening_components

['tagger', 'parser', 'senter']

In [21]:
tagger_tok2vec

{'@architectures': 'spacy.Tok2VecListener.v1',
 'width': '${components.tok2vec.model.encode:width}',
 'upstream': 'tok2vec'}

In [96]:
tok2vec.find_listeners()

TypeError: Tok2Vec.find_listeners() missing 1 required positional argument: 'component'

In [12]:
nlp._link_components()

In [16]:
nlp.config["components"]["senter"]

{'factory': 'senter',
 'model': {'@architectures': 'spacy.Tagger.v2',
  'nO': None,
  'normalize': False,
  'tok2vec': {'@architectures': 'spacy.Tok2Vec.v2',
   'embed': {'@architectures': 'spacy.MultiHashEmbed.v2',
    'width': 16,
    'attrs': ['NORM', 'PREFIX', 'SUFFIX', 'SHAPE', 'SPACY'],
    'rows': [1000, 500, 500, 500, 50],
    'include_static_vectors': False},
   'encode': {'@architectures': 'spacy.MaxoutWindowEncoder.v2',
    'width': 16,
    'depth': 2,
    'window_size': 1,
    'maxout_pieces': 2}}},
 'overwrite': False,
 'scorer': {'@scorers': 'spacy.senter_scorer.v1'}}

In [13]:
nlp.get_pipe_config("senter")["model"]["tok2vec"]

{'@architectures': 'spacy.Tok2Vec.v2',
 'embed': {'@architectures': 'spacy.MultiHashEmbed.v2',
  'width': 16,
  'attrs': ['NORM', 'PREFIX', 'SUFFIX', 'SHAPE', 'SPACY'],
  'rows': [1000, 500, 500, 500, 50],
  'include_static_vectors': False},
 'encode': {'@architectures': 'spacy.MaxoutWindowEncoder.v2',
  'width': 16,
  'depth': 2,
  'window_size': 1,
  'maxout_pieces': 2}}

In [94]:
nlp.get_pipe_config("senter")["model"]["tok2vec"]

{'@architectures': 'spacy.Tok2Vec.v2',
 'embed': {'@architectures': 'spacy.MultiHashEmbed.v2',
  'width': 16,
  'attrs': ['NORM', 'PREFIX', 'SUFFIX', 'SHAPE', 'SPACY'],
  'rows': [1000, 500, 500, 500, 50],
  'include_static_vectors': False},
 'encode': {'@architectures': 'spacy.MaxoutWindowEncoder.v2',
  'width': 16,
  'depth': 2,
  'window_size': 1,
  'maxout_pieces': 2}}

In [15]:
nlp.get_pipe_config("senter")["model"]

{'@architectures': 'spacy.Tagger.v2',
 'nO': None,
 'normalize': False,
 'tok2vec': {'@architectures': 'spacy.Tok2Vec.v2',
  'embed': {'@architectures': 'spacy.MultiHashEmbed.v2',
   'width': 16,
   'attrs': ['NORM', 'PREFIX', 'SUFFIX', 'SHAPE', 'SPACY'],
   'rows': [1000, 500, 500, 500, 50],
   'include_static_vectors': False},
  'encode': {'@architectures': 'spacy.MaxoutWindowEncoder.v2',
   'width': 16,
   'depth': 2,
   'window_size': 1,
   'maxout_pieces': 2}}}

> Note that running the text through the pipeline *before* modifying the `senter`'s `tok2vec` layer would also produce a `doc` with `sents`, but the results may differ as the `senter` component has not been trained with this `tok2vec` layer.

In [67]:
senter = nlp.get_pipe("senter")